# Test classification model against holdout data

### Imports

In [1]:
from keras import backend as K
from keras.models import load_model
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.optimizers import rmsprop
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Input, Dense
import os
import heapq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

Using TensorFlow backend.


### Functions

In [2]:
def get_genre(s):
    genre = s.split('/')[0]
    return genre

def get_song_id(s):
    id = s.split('__')[1]
    return id

def get_spect_num(s):
    spect = s.split('__')[2]
    return spect

def get_class_name(s):
    s = int(s.split('_')[1])
    return y[s]

def class_num_to_name(s):
    return y[s]

def convert_sum_of_prob(s):
    s = dict(s)
    return float(s['sum_of_prob'])

### Load model

In [5]:
prediction_data_dir = 'holdout_data/'
model = load_model('music_genre_classifier.hdf5')

In [7]:
num_classes = 9
image_size = 256
batch_size = 128

if K.image_data_format() == 'channels_first':
    input_shape = (3, image_size, image_size)
else:
    input_shape = (image_size, image_size, 3)

### Generate holdout predictions

In order to get a full understanding of the accuracy of the model I will need to sum the probabilities for all images belonging to a single track. For example, if there are 23 spectrogram images for a song, I need to collect all probabilities for all 23 images, then work out the 'full picture' by aggregating these probabilities and finding out what the top prediction was.

In [8]:
prediction_datagen = ImageDataGenerator(rescale=1./255)

prediction_generator = prediction_datagen.flow_from_directory(
    prediction_data_dir,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    shuffle=False,
    class_mode='categorical'
    )

Found 19266 images belonging to 9 classes.


In [9]:
prediction = model.predict_generator(prediction_generator, steps=(19266 // batch_size), verbose=1)

150/150 [==============================] - 179s     

In [10]:
labels = list(prediction_generator.class_indices.keys())

In [11]:
y = dict(zip(list(range(num_classes)), labels))

In [12]:
y

{0: 'breakbeat',
 1: 'dancehall_ragga',
 2: 'downtempo',
 3: 'drum_and_bass',
 4: 'funky_house',
 5: 'hip_hop_rnb',
 6: 'minimal_house',
 7: 'rock_indie',
 8: 'trance'}

Store all predictions for each image into a list:

In [13]:
all_predictions = []
for i in range(len(prediction)):
    p = heapq.nlargest(num_classes, enumerate(prediction[i]), key=lambda x: x[1])
    all_predictions.append(p)

In [14]:
# Get list of image file names
spect_file_names = []
for file in prediction_generator.filenames:
    spect_file_names.append(file)

In [15]:
# zip image file name with probability breakdowns
file_and_predictions = list(zip(spect_file_names, all_predictions))

In [16]:
file_and_predictions[0:1]

[('breakbeat/breakbeat__73191__000.png',
  [(0, 0.4868046),
   (3, 0.13068154),
   (1, 0.12670746),
   (5, 0.091813236),
   (4, 0.073000759),
   (2, 0.040806711),
   (6, 0.025267966),
   (7, 0.018976049),
   (8, 0.0059416713)])]

In [17]:
# transfer the data into a DataFrame and tidy up the data
tmp_df = pd.DataFrame(file_and_predictions)
tmp_df.columns = ['file_name','predictions']
tmp_df['actual_genre'] = tmp_df['file_name'].apply(get_genre)
tmp_df['song_id'] = tmp_df['file_name'].apply(get_song_id)
tmp_df['spect_num'] = tmp_df['file_name'].apply(get_spect_num)
tmp_df.drop('file_name', axis=1, inplace=True)

In [18]:
tmp_df.head()

,predictions,actual_genre,song_id,spect_num
0,"[(0, 0.486805), (3, 0.130682), (1, 0.126707), ...",breakbeat,73191,000.png
1,"[(0, 0.405314), (1, 0.35501), (5, 0.104498), (...",breakbeat,73191,001.png
2,"[(1, 0.629078), (0, 0.13335), (5, 0.129786), (...",breakbeat,73191,002.png
3,"[(1, 0.815709), (5, 0.146341), (0, 0.0230003),...",breakbeat,73191,003.png
4,"[(1, 0.482411), (4, 0.177705), (5, 0.171473), ...",breakbeat,73191,004.png


In [19]:
unique_song_ids = list(tmp_df['song_id'].unique())

In [20]:
unique_song_ids[0:6]

['73191', '74093', '74109', '74299', '74307', '74311']

Loop through each track in the dataframe and store sum of class predictions in a dictionary.

In [21]:
song_predictions_dict = {}

In [22]:
for song in unique_song_ids:
    l = list(tmp_df[tmp_df['song_id'] == song]['predictions'])
    l = [item for sublist in l for item in sublist]
    
    # creates a dictionary of all the probability values for each genre for a specific track
    class_probs = dict()
    for f in l:
        class_num = f[0]
        class_name = class_num_to_name(class_num)
        class_prob = f[1]
        if class_name in class_probs:
            class_probs[class_name].append(class_prob)
        else:
            class_probs[class_name] = [class_prob]
    
    # now store the sums of this data into the song_predictions_dict using song_id as the key
    genre_dict = {}
    for genre in class_probs.keys():
        d = {
            'sum_of_prob':sum(class_probs[genre])
            }
        
        genre_dict[genre] = d
    
    song_predictions_dict[song] = genre_dict

In [23]:
song_predictions_dict['74093']

{'breakbeat': {'sum_of_prob': 15.485807754099369},
 'dancehall_ragga': {'sum_of_prob': 0.40356463479656668},
 'downtempo': {'sum_of_prob': 0.27277979874634184},
 'drum_and_bass': {'sum_of_prob': 1.0843666558503173},
 'funky_house': {'sum_of_prob': 1.4116746964864433},
 'hip_hop_rnb': {'sum_of_prob': 0.86915496045548934},
 'minimal_house': {'sum_of_prob': 0.3322015489829937},
 'rock_indie': {'sum_of_prob': 0.50900403872947209},
 'trance': {'sum_of_prob': 2.6314460313878953}}

In [24]:
song_predictions_dict['73191']

{'breakbeat': {'sum_of_prob': 6.2423023153096437},
 'dancehall_ragga': {'sum_of_prob': 7.4230044297873974},
 'downtempo': {'sum_of_prob': 0.93571153946686536},
 'drum_and_bass': {'sum_of_prob': 2.1506282689515501},
 'funky_house': {'sum_of_prob': 1.0640672014851589},
 'hip_hop_rnb': {'sum_of_prob': 4.0336373886093497},
 'minimal_house': {'sum_of_prob': 0.17466602692729793},
 'rock_indie': {'sum_of_prob': 0.71399763729277765},
 'trance': {'sum_of_prob': 0.26198558195028454}}

In [25]:
df_sum_probs = pd.DataFrame.from_dict(song_predictions_dict, orient='index').reset_index()

In [26]:
df_sum_probs.head()

,index,breakbeat,drum_and_bass,dancehall_ragga,hip_hop_rnb,funky_house,downtempo,minimal_house,rock_indie,trance
0,428650,{'sum_of_prob': 4.10746110138},{'sum_of_prob': 0.41520630123},{'sum_of_prob': 8.42639565794},{'sum_of_prob': 4.57485647779},{'sum_of_prob': 1.34636493772},{'sum_of_prob': 0.358994856026},{'sum_of_prob': 0.222413361305},{'sum_of_prob': 1.30894568554},{'sum_of_prob': 2.23936167749}
1,428652,{'sum_of_prob': 0.170717336528},{'sum_of_prob': 0.0582605770469},{'sum_of_prob': 7.75840958208},{'sum_of_prob': 14.7442003936},{'sum_of_prob': 0.091962097762},{'sum_of_prob': 0.100306822322},{'sum_of_prob': 0.0243674241547},{'sum_of_prob': 0.0500449305691},{'sum_of_prob': 0.00173062102579}
2,428656,{'sum_of_prob': 0.819452046766},{'sum_of_prob': 0.074765845301},{'sum_of_prob': 10.7249140143},{'sum_of_prob': 10.6443867087},{'sum_of_prob': 0.352652344445},{'sum_of_prob': 0.0913896150232},{'sum_of_prob': 0.0548552668261},{'sum_of_prob': 0.222235039691},{'sum_of_prob': 0.0153485667097}
3,428659,{'sum_of_prob': 0.255360172887},{'sum_of_prob': 0.611347321457},{'sum_of_prob': 12.1666831747},{'sum_of_prob': 6.03037010878},{'sum_of_prob': 0.305300723034},{'sum_of_prob': 2.25140022545},{'sum_of_prob': 0.153904794421},{'sum_of_prob': 1.18166395184},{'sum_of_prob': 0.0439688810457}
4,428664,{'sum_of_prob': 0.785700174747},{'sum_of_prob': 0.125477448804},{'sum_of_prob': 5.90105446428},{'sum_of_prob': 12.036060378},{'sum_of_prob': 0.278910861351},{'sum_of_prob': 0.526555568445},{'sum_of_prob': 0.0651680387964},{'sum_of_prob': 3.25425500516},{'sum_of_prob': 0.0268181300016}


In [27]:
df_sum_probs['breakbeat'] = df_sum_probs['breakbeat'].apply(convert_sum_of_prob)
df_sum_probs['drum_and_bass'] = df_sum_probs['drum_and_bass'].apply(convert_sum_of_prob)
df_sum_probs['dancehall_ragga'] = df_sum_probs['dancehall_ragga'].apply(convert_sum_of_prob)
df_sum_probs['hip_hop_rnb'] = df_sum_probs['hip_hop_rnb'].apply(convert_sum_of_prob)
df_sum_probs['funky_house'] = df_sum_probs['funky_house'].apply(convert_sum_of_prob)
df_sum_probs['downtempo'] = df_sum_probs['downtempo'].apply(convert_sum_of_prob)
df_sum_probs['minimal_house'] = df_sum_probs['minimal_house'].apply(convert_sum_of_prob)
df_sum_probs['rock_indie'] = df_sum_probs['rock_indie'].apply(convert_sum_of_prob)
df_sum_probs['trance'] = df_sum_probs['trance'].apply(convert_sum_of_prob)
df_sum_probs = df_sum_probs.rename(columns={'index':'song_id'})

In [28]:
df_sum_probs.head(5)

,song_id,breakbeat,drum_and_bass,dancehall_ragga,hip_hop_rnb,funky_house,downtempo,minimal_house,rock_indie,trance
0,428650,4.107461,0.415206,8.426396,4.574856,1.346365,0.358995,0.222413,1.308946,2.239362
1,428652,0.170717,0.058261,7.758410,14.744200,0.091962,0.100307,0.024367,0.050045,0.001731
2,428656,0.819452,0.074766,10.724914,10.644387,0.352652,0.091390,0.054855,0.222235,0.015349
3,428659,0.255360,0.611347,12.166683,6.030370,0.305301,2.251400,0.153905,1.181664,0.043969
4,428664,0.785700,0.125477,5.901054,12.036060,0.278911,0.526556,0.065168,3.254255,0.026818


Now I have a DataFrame containing the song_id and the respective sum of probabilities for all spectrograms belonging to that song.

In [29]:
df_sum_probs['total_sum']= df_sum_probs.iloc[:, -9:].sum(axis=1)

In [30]:
df_sum_probs.head(5)

,song_id,breakbeat,drum_and_bass,dancehall_ragga,hip_hop_rnb,funky_house,downtempo,minimal_house,rock_indie,trance,total_sum
0,428650,4.107461,0.415206,8.426396,4.574856,1.346365,0.358995,0.222413,1.308946,2.239362,23.000000
1,428652,0.170717,0.058261,7.758410,14.744200,0.091962,0.100307,0.024367,0.050045,0.001731,23.000000
2,428656,0.819452,0.074766,10.724914,10.644387,0.352652,0.091390,0.054855,0.222235,0.015349,22.999999
3,428659,0.255360,0.611347,12.166683,6.030370,0.305301,2.251400,0.153905,1.181664,0.043969,22.999999
4,428664,0.785700,0.125477,5.901054,12.036060,0.278911,0.526556,0.065168,3.254255,0.026818,23.000000


If I divide these numbers by the total sum, I'll get the probabilities over all images belonging to a track

In [31]:
df_sum_probs['breakbeat']= df_sum_probs['breakbeat'] / df_sum_probs['total_sum']
df_sum_probs['drum_and_bass']= df_sum_probs['drum_and_bass'] / df_sum_probs['total_sum']
df_sum_probs['dancehall_ragga']= df_sum_probs['dancehall_ragga'] / df_sum_probs['total_sum']
df_sum_probs['hip_hop_rnb']= df_sum_probs['hip_hop_rnb'] / df_sum_probs['total_sum']
df_sum_probs['funky_house']= df_sum_probs['funky_house'] / df_sum_probs['total_sum']
df_sum_probs['downtempo']= df_sum_probs['downtempo'] / df_sum_probs['total_sum']
df_sum_probs['minimal_house']= df_sum_probs['minimal_house'] / df_sum_probs['total_sum']
df_sum_probs['rock_indie']= df_sum_probs['rock_indie'] / df_sum_probs['total_sum']
df_sum_probs['trance']= df_sum_probs['trance'] / df_sum_probs['total_sum']
df_sum_probs.drop('total_sum', axis=1, inplace=True)

In [32]:
df_sum_probs.head(5)

,song_id,breakbeat,drum_and_bass,dancehall_ragga,hip_hop_rnb,funky_house,downtempo,minimal_house,rock_indie,trance
0,428650,0.178585,0.018052,0.366365,0.198907,0.058538,0.015608,0.009670,0.056911,0.097364
1,428652,0.007422,0.002533,0.337322,0.641052,0.003998,0.004361,0.001059,0.002176,0.000075
2,428656,0.035628,0.003251,0.466301,0.462799,0.015333,0.003973,0.002385,0.009662,0.000667
3,428659,0.011103,0.026580,0.528986,0.262190,0.013274,0.097887,0.006692,0.051377,0.001912
4,428664,0.034161,0.005456,0.256568,0.523307,0.012127,0.022894,0.002833,0.141489,0.001166


In [33]:
actual_genres = tmp_df[['actual_genre','song_id']].drop_duplicates()

Calculate the top prediction for each song:

In [34]:
full_breakdown = actual_genres.merge(df_sum_probs)
full_breakdown['first_prediction'] = full_breakdown[['breakbeat','drum_and_bass','dancehall_ragga',
                                                     'hip_hop_rnb','funky_house','downtempo',
                                                     'minimal_house','rock_indie','trance']].idxmax(axis=1)

In [35]:
full_breakdown.head()

,actual_genre,song_id,breakbeat,drum_and_bass,dancehall_ragga,hip_hop_rnb,funky_house,downtempo,minimal_house,rock_indie,trance,first_prediction
0,breakbeat,73191,0.271404,0.093506,0.322739,0.175376,0.046264,0.040683,0.007594,0.031043,0.011391,dancehall_ragga
1,breakbeat,74093,0.673296,0.047146,0.017546,0.037789,0.061377,0.011860,0.014444,0.022131,0.114411,breakbeat
2,breakbeat,74109,0.020109,0.011923,0.122112,0.033048,0.267638,0.329403,0.108211,0.081416,0.026140,downtempo
3,breakbeat,74299,0.301416,0.023911,0.155262,0.180262,0.117227,0.106337,0.019794,0.039665,0.056126,breakbeat
4,breakbeat,74307,0.242033,0.017999,0.112116,0.128351,0.180337,0.170797,0.066654,0.034132,0.047582,breakbeat


Determine whether the prediction was correct:

In [36]:
full_breakdown['correct_prediction'] = full_breakdown['actual_genre'] == full_breakdown['first_prediction']

In [37]:
full_breakdown.head()

,actual_genre,song_id,breakbeat,drum_and_bass,dancehall_ragga,hip_hop_rnb,funky_house,downtempo,minimal_house,rock_indie,trance,first_prediction,correct_prediction
0,breakbeat,73191,0.271404,0.093506,0.322739,0.175376,0.046264,0.040683,0.007594,0.031043,0.011391,dancehall_ragga,False
1,breakbeat,74093,0.673296,0.047146,0.017546,0.037789,0.061377,0.011860,0.014444,0.022131,0.114411,breakbeat,True
2,breakbeat,74109,0.020109,0.011923,0.122112,0.033048,0.267638,0.329403,0.108211,0.081416,0.026140,downtempo,False
3,breakbeat,74299,0.301416,0.023911,0.155262,0.180262,0.117227,0.106337,0.019794,0.039665,0.056126,breakbeat,True
4,breakbeat,74307,0.242033,0.017999,0.112116,0.128351,0.180337,0.170797,0.066654,0.034132,0.047582,breakbeat,True


In [38]:
len(full_breakdown)

866

In [39]:
len(full_breakdown[full_breakdown['correct_prediction'] == True])

648

Examine the breakdown by genre:

In [42]:
full_breakdown = full_breakdown.groupby(['actual_genre','correct_prediction'])['song_id'].count().reset_index()

In [45]:
full_breakdown

,actual_genre,correct_prediction,song_id
0,breakbeat,False,22
1,breakbeat,True,76
2,dancehall_ragga,False,21
3,dancehall_ragga,True,79
4,downtempo,False,28
5,downtempo,True,69
6,drum_and_bass,False,10
7,drum_and_bass,True,88
8,funky_house,False,28
9,funky_house,True,69


### Classification accuracy by genre

In [53]:
(full_breakdown.groupby('actual_genre')['song_id'].max() / \
    full_breakdown.groupby('actual_genre')['song_id'].sum()).reset_index()

,actual_genre,song_id
0,breakbeat,0.775510
1,dancehall_ragga,0.790000
2,downtempo,0.711340
3,drum_and_bass,0.897959
4,funky_house,0.711340
5,hip_hop_rnb,0.610000
6,minimal_house,0.626263
7,rock_indie,0.703704
8,trance,0.906250
